### 0. import packages

In [1]:
import time
from openai import OpenAI
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Fetch the API key from the environment variable
api_key = os.getenv('OPENAI_API_KEY')

### 1. System Prompt

In [2]:
# Define system prompt
system_prompt = {
    "role": "system",
    "content": '''
    <Basic>
        1) 사용자 질문(Question)에 따라 핵심 정보를 추출하기 위해 키워드, 즉 슬롯을 추출하기 위한 프롬프트를 작성하고 싶어.
        2) 다음과 같은 카테고리들을 "슬롯"이라고 명칭할 것이고 슬롯 목록은 다음과 같아: "개인화요소", "핵심 키워드", "감정/분위기", "시기/상황", "독서 목적", "대상 독자", "난이도", "분량"
    </Basic>
    <Check>
        사용자 질문에서 핵심 정보를 추출하기 위해 다음 단계를 따를 것:
        1) 질문의 핵심 주제나 의도를 파악하여 요청된 조건과 필요한 정보를 분리
        2) 의미를 명확하게 전달하는 중요 단어를 선택하여 각 슬롯에 적합한 키워드를 추출
            2-1. 질문에서 의미를 명확하게 전달하는 주요 명사, 형용사, 동사를 선택하여 적절한 슬롯에 매칭
            2-2. 불필요한 연결어, 조사(은, 는, 이, 가 등) 및 일반적인 표현(예: 좋은 책)은 배제
        3) 추출된 키워드는 각 슬롯 목록과 매칭되도록 구조화해서 정리(슬롯에 들어갈만한 쿼리값을 JSON형식으로 구조화하여 전달)
            3-1. "개인화요소"는 메타정보와 매칭되는 경우 'Y', 아닌 경우는 'N'으로 출력하며 매칭되는 경우 해당 정보를 리스트 형식으로 제공
            3-2. "핵심 키워드"는 사용자 질문 및 "개인화요소", "감정/분위기", "시기/상황", "독서 목적", "대상 독자", "난이도", "분량" 등의 결과물을 고려하여 대표성을 띄는 키워드로 출력
            3-3. "감정/분위기"의 경우 따뜻한 톤, 활기찬 톤, 긴장감 등의 다양한 상황을 고려하여 적절한 문맥으로 출력
            3-4. "시기/상황"의 경우 오늘은 2025-02-10이며, 출간일 제한이 필요한 경우 yyyy-mm-dd 형태와 이전_이후 여부로 출력
            3-5. "독서 목적"의 경우 안정, 오락, 자기계발, 지식탐구 등의 다양한 카테고리로 나눠서 반영
            3-6. "대상 독자"의 경우 연령별, 성별, 특정 직업군, 관심사 등의 유형으로 세분화하여 반영
            3-7. "난이도"의 경우 1부터 5단계로 정의, 1에서 5로 갈수록 난이도가 올라가는 것으로 간주
            3-8. "분량"의 경우 평균적인 일반단행본 페이지수를 고려하여 적절한 int값으로 제공(너무 짧은 분량이면 대상 도서가 없을 수도 있으니, 단편집 같은 적절한 포맷도 카테고리로 고려)

        4) 3-1부터 3-8까지 매칭되지 않는 필드는 일단 NA로 반환한 후 <Notes> 조건을 반영해줘
    </Check>
    <Notes>
        1) 매칭되지 않는 필드, 즉 "NA"로 반환되는 슬롯 값의 경우 값 또는 키값 모두 출력되지 않을 것
        2) 날짜 포맷과 난이도 레벨, 페이지 수는 반드시 명시된 기준에 맞출 것
        3) JSON은 참고할 부분을 키 값으로 해서 하나로 출력할 것
        4) "핵심 키워드"의 경우 대표성을 띄는 키워드로 출력하되 중복되지 않도록 출력할 것
    </Notes>
    <Output>
        질문에 대한 슬롯 매핑의 예시는 다음과 같습니다:

        {
        "개인화요소": "N",
        "핵심 키워드": ["풍수", "멘탈", "신비주의", "과거 성찰", "학습 효율"],
        "감정/분위기": ["따뜻한", "잔잔한", "밝은", "신나는", "불안한", "서늘한"],
        "시기/상황": {
            "이전_이후": "이후",
            "기준일": "2024-11-07"
        },
        "독서 목적": ["힐링", "재미", "안정", "자기계발", "학습"],
        "대상 독자": ["20대", "전공자", "프리랜서", "요리"],
        "난이도": 2,
        "분량": {
            "최소_페이지": 50,
            "최대_페이지": 120
        }
        }
    '''
}

### 2. Define function

In [3]:
# Function to create messages for the OpenAI API
def create_messages(user_messages):
    return [system_prompt] + [{"role": "user", "content": msg} for msg in user_messages]

# Define function to test with multiple models
def test_models(models):
    results = {}
    for model_name in models:
        # Set the model in the environment
        os.environ['LLM_MODEL'] = model_name
        
        # Create OpenAI client with current model
        client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])
        
        # Create OpenAI messages
        current_messages = create_messages(user_messages)

        # Record start time
        start_time = time.time()
        
        # Make API call
        response = client.chat.completions.create(
            model=model_name,
            messages=current_messages,
            stream=False
        )
        
        # Record end time and calculate duration
        end_time = time.time()
        elapsed_time = round(end_time - start_time, 2)
        
        # Store the result and elapsed time
        results[model_name] = {
            "output": response.choices[0].message.content,
            "time_taken": f"{elapsed_time} seconds"
        }

    return results


### 3. Output

In [4]:
# Test user input
user_messages = ['30분만에 읽을 만한 책 중 최근 3개월 이내 출판된 책으로 요즘 같은 추운 날씨에 읽기 좋은 책 추천해줘.']

# Models to test
models_to_test = ['gpt-4o-mini', 'gpt-4o', 'o3-mini']

# Test and print results
output_results = test_models(models_to_test)

# Display results for each model
for model, result in output_results.items():
    print(f"### Output for model {model} ###\n")
    print(f"Time taken: {result['time_taken']}\n")
    print(result['output'])
    print("\n" + "="*50 + "\n")


### Output for model gpt-4o-mini ###

Time taken: 4.5 seconds

{
    "개인화요소": "N",
    "핵심 키워드": ["책", "최근 출판", "30분", "추운 날씨", "추천"],
    "감정/분위기": ["따뜻한", "편안한", "잔잔한"],
    "시기/상황": {
        "이전_이후": "이전",
        "기준일": "2025-02-10"
    },
    "독서 목적": ["오락", "힐링"],
    "대상 독자": ["모든 연령"],
    "난이도": 1,
    "분량": {
        "최소_페이지": 20,
        "최대_페이지": 50
    }
}


### Output for model gpt-4o ###

Time taken: 2.84 seconds

{
    "핵심 키워드": ["짧은 독서", "최근 출판", "추운 날씨"],
    "시기/상황": {
        "이전_이후": "이후",
        "기준일": "2024-11-10"
    },
    "분량": {
        "최소_페이지": 50,
        "최대_페이지": 80
    }
}


### Output for model o3-mini ###

Time taken: 17.18 seconds

{
  "개인화요소": "N",
  "핵심 키워드": ["30분", "최근3개월", "추운날씨", "짧은책"],
  "감정/분위기": ["따뜻한"],
  "시기/상황": {
    "이전_이후": "이후",
    "기준일": "2024-11-10"
  },
  "독서 목적": ["힐링"],
  "분량": {
    "최소_페이지": 30,
    "최대_페이지": 70
  }
}




### 4. 기타
* 사용자 질문 10개 출력 결과 비교

In [10]:
import time
from openai import OpenAI
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Initialize OpenAI client globally (no need to recreate per model)
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

# Function to create messages for the OpenAI API
def create_messages(user_message):
    return [system_prompt] + [{"role": "user", "content": user_message}]

# Define function to test with multiple models for all user messages
def test_models(models, user_messages):
    results = {question: {} for question in user_messages}  # Initialize result storage
    
    # Loop through each user message
    for user_message in user_messages:
        for model_name in models:
            # Create OpenAI messages for the current user message
            current_messages = create_messages(user_message)

            # Record start time
            start_time = time.time()

            # Make API call with dynamically selected model
            response = client.chat.completions.create(
                model=model_name,
                messages=current_messages,
                stream=False
            )

            # Record end time and calculate duration
            end_time = time.time()
            elapsed_time = round(end_time - start_time, 2)

            # Store result using correct response access
            message_content = response.choices[0].message.content

            results[user_message][model_name] = {
                "output": message_content,
                "time_taken": f"{elapsed_time} seconds"
            }
    
    return results

# Test user input
user_messages = [
    "책을 출판하려면 어떻게 해야하는지 혹시 전반적인 지식을 다루는 책이 있을까? 내가 글을 쓰는 것부터 출판하는 것까지를 해보고 싶은데 아는 게 없어.",
    "역사에 대해서 하나도 모르는데 최근 방송을 보니까 공부해보고 싶어졌어. 초보도 쉽게 이해할만한 책으로 추천해줘.",
    "ML/DL(기계 학습/딥 러닝)을 공부하고 싶어. 어떤 커리큘럼이 좋을까? 딥 러닝을 공부랑 때 가장 도움이 되는 책을 단계별로 알려줘.",
    "자신감이 부족한데, 자기 자신을 사랑하는 법에 대해 다룬 책 뭐가 있을까?",
    "중년기를 다룬 자서전을 추천해줘. 단순한 성장 스토리만이 아니라, 당시의 사회적 가치나 관습을 반영한 작품이 좋겠어. 예를 들어 『힐빌리의 노래』나 처칠의 『윈스턴 처칠, 나의 청춘』 같은 것들 말이야.",
    "조카에게 선물할 책을 찾고 있어요. 여자아이고 여섯살인데 어떤 책을 사주어야 할 지 모르겠어요.",
    "지구에서 한아뿐이랑 아가미 파과 아몬드 버드스트라이크 보건교사 안은영은 읽었어. 얇고 가벼우면서 술술 읽히는 책 추천 좀 해줄래?",
    "책을 자주 읽는 습관을 기르고 싶은데, 도움될 만한 책 있어?",
    "생일 축하합니다라는 말이 많이 나오는 책을 알려줄 수 있어?",
    "어톤먼트 책 표지가 마음에 안드는데, 다른 에디션도 있을까?"
]

# Models to test
models_to_test = ['gpt-4o-mini', 'gpt-4o', 'o3-mini']

# Execute the model testing
output_results = test_models(models_to_test, user_messages)

# Display the results
for question, model_outputs in output_results.items():
    print(f"\n### 질문: {question} ###\n")
    for model, result in model_outputs.items():
        print(f"모델: {model.upper()}")
        print(f"응답 시간: {result['time_taken']}")
        print(f"응답 내용:\n{result['output']}\n")
        print("-" * 50)



### 질문: 책을 출판하려면 어떻게 해야하는지 혹시 전반적인 지식을 다루는 책이 있을까? 내가 글을 쓰는 것부터 출판하는 것까지를 해보고 싶은데 아는 게 없어. ###

모델: GPT-4O-MINI
응답 시간: 3.42 seconds
응답 내용:
{
    "개인화요소": "N",
    "핵심 키워드": ["출판", "저자", "글쓰기", "전반적인 지식"],
    "감정/분위기": ["신나는", "불안한", "도전적인"],
    "시기/상황": {
        "이전_이후": "이후",
        "기준일": "2025-02-10"
    },
    "독서 목적": ["자기계발", "지식탐구", "실용", "정보 습득"],
    "대상 독자": ["초보 작가", "관심 있는 일반인"],
    "난이도": 2,
    "분량": {
        "최소_페이지": 150,
        "최대_페이지": 300
    }
}

--------------------------------------------------
모델: GPT-4O
응답 시간: 2.36 seconds
응답 내용:
{
    "개인화요소": "N",
    "핵심 키워드": ["책 출판", "전반적인 지식", "글쓰기", "출판 과정"],
    "독서 목적": ["자기계발", "지식탐구"],
    "대상 독자": ["일반 독자"],
    "난이도": 3
}

--------------------------------------------------
모델: O3-MINI
응답 시간: 15.66 seconds
응답 내용:
{
  "개인화요소": "N",
  "핵심 키워드": [
    "출판",
    "글쓰기",
    "전반적 지식"
  ],
  "감정/분위기": [
    "불안한"
  ],
  "독서 목적": [
    "자기계발"
  ],
  "대상 독자": [
    "예비 작가"
  ],
  "난이도": 1,
  "분량": {
    "최소_페이지": 150